In [66]:
import pandas as pd
import numpy as np

# it was written on the medium to Refer to :- https://www.kaggle.com/rinnqd/reduce-memory-usage

def reduce_memory_usage(df):
  
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [7]:
train_data = reduce_memory_usage(pd.read_csv("application_train.csv"))
bureau_data = reduce_memory_usage(pd.read_csv('bureau.csv'))
bureau_balance = reduce_memory_usage(pd.read_csv('bureau_balance.csv'))
prev_app = reduce_memory_usage(pd.read_csv('previous_application.csv'))
installments = reduce_memory_usage(pd.read_csv('installments_payments.csv'))
pos_cash = reduce_memory_usage(pd.read_csv('POS_CASH_balance.csv'))
credit_card = reduce_memory_usage(pd.read_csv('credit_card_balance.csv'))

print("PRINTING OUT ALL FEATURES IN THE DATASET")
print("\nFeatures/Columns in train_data include: \n", train_data.columns.tolist())
print("\nFeatures/Columns in bureau_data include: \n", bureau_data.columns.tolist())
print("\nFeatures/Columns in bureau_balance include: \n", bureau_balance.columns.tolist())
print("\nFeatures/Columns in prev_app include: \n", prev_app.columns.tolist())
print("\nFeatures/Columns in installments include: \n", installments.columns.tolist())
print("\nFeatures/Columns in pos_cash include: \n", pos_cash.columns.tolist())
print("\nFeatures/Columns in credit_card include: \n", credit_card.columns.tolist())

Memory usage of dataframe is 286.23 MB
Memory usage after optimization is: 92.38 MB
Decreased by 67.7%
Memory usage of dataframe is 222.62 MB
Memory usage after optimization is: 112.95 MB
Decreased by 49.3%
Memory usage of dataframe is 624.85 MB
Memory usage after optimization is: 338.46 MB
Decreased by 45.8%
Memory usage of dataframe is 471.48 MB
Memory usage after optimization is: 309.01 MB
Decreased by 34.5%
Memory usage of dataframe is 830.41 MB
Memory usage after optimization is: 311.40 MB
Decreased by 62.5%
Memory usage of dataframe is 610.43 MB
Memory usage after optimization is: 238.45 MB
Decreased by 60.9%
Memory usage of dataframe is 673.88 MB
Memory usage after optimization is: 289.33 MB
Decreased by 57.1%
PRINTING OUT ALL FEATURES IN THE DATASET

Features/Columns in train_data include: 
 ['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_TY

In [68]:
print("Loading datasets...")
bureau = reduce_memory_usage(pd.read_csv('bureau.csv'))
bureau_balance = reduce_memory_usage(pd.read_csv('bureau_balance.csv'))
prev_app = reduce_memory_usage(pd.read_csv('previous_application.csv'))
installments = reduce_memory_usage(pd.read_csv('installments_payments.csv'))
credit_card = reduce_memory_usage(pd.read_csv('credit_card_balance.csv'))
application_train = reduce_memory_usage(pd.read_csv('application_train.csv'))

# -------------------------------------------------------------------
# 1. CREDIT BUREAU HISTORY (from bureau + bureau_balance)
# -------------------------------------------------------------------
print("Creating balance_history.csv...")

# Merge bureau with its balance data
status_agg = pd.get_dummies(bureau_balance[['SK_ID_BUREAU', 'STATUS']], columns=['STATUS'])
status_agg = status_agg.groupby('SK_ID_BUREAU', as_index=False).sum()

# Create clean dataset
balance_history = bureau.merge(
    status_agg, 
    on='SK_ID_BUREAU', 
    how='left'
)[[
    'SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY',
    'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'AMT_CREDIT_SUM',
    'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_OVERDUE',
    'STATUS_0', 'STATUS_1', 'STATUS_2', 'STATUS_3', 'STATUS_4', 'STATUS_5', 'STATUS_C', 'STATUS_X'
]].rename(columns={
    'SK_ID_CURR': 'customer_id',
    'SK_ID_BUREAU': 'credit_bureau_id',
    'CREDIT_ACTIVE': 'credit_status',
    'CREDIT_CURRENCY': 'credit_currency',
    'DAYS_CREDIT': 'days_since_credit_start',
    'CREDIT_DAY_OVERDUE': 'current_overdue_days',
    'AMT_CREDIT_SUM': 'total_credit_amount',
    'AMT_CREDIT_SUM_DEBT': 'current_debt_amount',
    'AMT_CREDIT_SUM_OVERDUE': 'current_overdue_amount',
    'STATUS_0': 'months_on_time',
    'STATUS_1': 'months_1_30_dpd',
    'STATUS_2': 'months_31_60_dpd',
    'STATUS_3': 'months_61_90_dpd',
    'STATUS_4': 'months_91_120_dpd',
    'STATUS_5': 'months_120_plus_dpd',
    'STATUS_C': 'months_closed',
    'STATUS_X': 'months_no_info'
})

balance_history.to_csv('balance_history.csv', index=False)

# -------------------------------------------------------------------
# 2. CREDIT CARD TRANSACTIONS (from credit_card_balance)
# -------------------------------------------------------------------
print("Creating transaction_history.csv...")

transaction_history = credit_card[[
    'SK_ID_CURR', 'SK_ID_PREV', 'MONTHS_BALANCE',
    'AMT_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL',
    'AMT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_CURRENT',
    'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT',
    'SK_DPD', 'SK_DPD_DEF'
]].rename(columns={
    'SK_ID_CURR': 'customer_id',
    'SK_ID_PREV': 'previous_loan_id',
    'MONTHS_BALANCE': 'months_since_transaction',
    'AMT_BALANCE': 'current_balance',
    'AMT_CREDIT_LIMIT_ACTUAL': 'credit_limit',
    'AMT_DRAWINGS_CURRENT': 'total_spent_amount',
    'CNT_DRAWINGS_CURRENT': 'transaction_count',
    'AMT_PAYMENT_CURRENT': 'minimum_payment_amount',
    'AMT_PAYMENT_TOTAL_CURRENT': 'total_payment_amount',
    'SK_DPD': 'days_past_due',
    'SK_DPD_DEF': 'days_past_due_90_plus'
})

transaction_history.to_csv('transaction_history.csv', index=False)

# -------------------------------------------------------------------
# 3. PREVIOUS LOAN APPLICATIONS (from previous_application)
# -------------------------------------------------------------------
print("Creating loan_application_history.csv...")

# Preserve TARGET from original training data
target_map = application_train[['SK_ID_CURR', 'TARGET']].set_index('SK_ID_CURR')['TARGET']

loan_application_history = prev_app[[
    'SK_ID_CURR', 'SK_ID_PREV', 'NAME_CONTRACT_TYPE',
    'AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT',
    'DAYS_DECISION', 'NAME_CONTRACT_STATUS',
    'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON'
]].rename(columns={
    'SK_ID_CURR': 'customer_id',
    'SK_ID_PREV': 'previous_loan_id',
    'NAME_CONTRACT_TYPE': 'loan_type',
    'AMT_ANNUITY': 'annuity_amount',
    'AMT_APPLICATION': 'requested_amount',
    'AMT_CREDIT': 'approved_amount',
    'DAYS_DECISION': 'days_since_decision',
    'NAME_CONTRACT_STATUS': 'application_status',
    'NAME_PAYMENT_TYPE': 'payment_type',
    'CODE_REJECT_REASON': 'reject_reason_code'
})

# Add repayment capability flag
loan_application_history['repayment_capability'] = loan_application_history['customer_id'].map(target_map)
loan_application_history.to_csv('loan_application_history.csv', index=False)

# -------------------------------------------------------------------
# 4. LOAN REPAYMENT HISTORY (from installments_payments)
# -------------------------------------------------------------------
print("Creating loan_repayment_history.csv...")

loan_repayment_history = installments[[
    'SK_ID_CURR', 'SK_ID_PREV', 
    'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT',
    'AMT_INSTALMENT', 'AMT_PAYMENT'
]].rename(columns={
    'SK_ID_CURR': 'customer_id',
    'SK_ID_PREV': 'previous_loan_id',
    'DAYS_INSTALMENT': 'due_date_days',
    'DAYS_ENTRY_PAYMENT': 'payment_date_days',
    'AMT_INSTALMENT': 'due_amount',
    'AMT_PAYMENT': 'paid_amount'
})

loan_repayment_history.to_csv('loan_repayment_history.csv', index=False)

# -------------------------------------------------------------------
# 5. CUSTOMER DEMOGRAPHICS (from application_train)
# -------------------------------------------------------------------
print("Creating customer_bio_data.csv...")

customer_bio_data = application_train[[
    'SK_ID_CURR', 'CODE_GENDER', 
    'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 
    'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 
    'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS',
    'DAYS_BIRTH', 'DAYS_EMPLOYED', 'OCCUPATION_TYPE',
    'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3'
]].rename(columns={
    'SK_ID_CURR': 'customer_id',
    'CODE_GENDER': 'gender',
    'FLAG_OWN_CAR': 'owns_car_flag',
    'FLAG_OWN_REALTY': 'owns_property_flag',
    'CNT_CHILDREN': 'children_count',
    'AMT_INCOME_TOTAL': 'annual_income',
    'NAME_EDUCATION_TYPE': 'education_level',
    'NAME_FAMILY_STATUS': 'family_status',
    'DAYS_BIRTH': 'age_in_days',
    'DAYS_EMPLOYED': 'employment_days',
    'OCCUPATION_TYPE': 'occupation',
    'EXT_SOURCE_1': 'external_score_1',
    'EXT_SOURCE_2': 'external_score_2',
    'EXT_SOURCE_3': 'external_score_3'
})

customer_bio_data.to_csv('customer_bio_data.csv', index=False)

print("All 5 clean datasets created successfully!")

Loading datasets...
Memory usage of dataframe is 222.62 MB
Memory usage after optimization is: 112.95 MB
Decreased by 49.3%
Memory usage of dataframe is 624.85 MB
Memory usage after optimization is: 338.46 MB
Decreased by 45.8%
Memory usage of dataframe is 471.48 MB
Memory usage after optimization is: 309.01 MB
Decreased by 34.5%
Memory usage of dataframe is 830.41 MB
Memory usage after optimization is: 311.40 MB
Decreased by 62.5%
Memory usage of dataframe is 673.88 MB
Memory usage after optimization is: 289.33 MB
Decreased by 57.1%
Memory usage of dataframe is 286.23 MB
Memory usage after optimization is: 92.38 MB
Decreased by 67.7%
Creating balance_history.csv...
Creating transaction_history.csv...
Creating loan_application_history.csv...
Creating loan_repayment_history.csv...
Creating customer_bio_data.csv...
All 5 clean datasets created successfully!


In [69]:
print("LOADING NEW DATASET..")
balance_history = reduce_memory_usage(pd.read_csv("balance_history.csv"))
transaction_history = reduce_memory_usage(pd.read_csv("transaction_history.csv"))
loan_application_history = reduce_memory_usage(pd.read_csv("loan_application_history.csv"))
loan_repayment_history = reduce_memory_usage(pd.read_csv("loan_repayment_history.csv"))
customer_bio_data = reduce_memory_usage(pd.read_csv("customer_bio_data.csv"))

print("PRINTING OUT ALL NEW FEATURES IN THE DATASET")
print("\nFeatures/Columns in balance_history include: ", balance_history.columns.tolist())
print("\nFeatures/Columns in transaction_history include: ", transaction_history.columns.tolist())
print("\nFeatures/Columns in loan_application_history include: ", loan_application_history.columns.tolist())
print("\nFeatures/Columns in loan_repayment_history include: ", loan_repayment_history.columns.tolist())
print("\nFeatures/Columns in customer_bio_data include: ", customer_bio_data.columns.tolist())

LOADING NEW DATASET..
Memory usage of dataframe is 222.62 MB
Memory usage after optimization is: 91.67 MB
Decreased by 58.8%
Memory usage of dataframe is 322.29 MB
Memory usage after optimization is: 128.18 MB
Decreased by 60.2%
Memory usage of dataframe is 140.17 MB
Memory usage after optimization is: 89.20 MB
Decreased by 36.4%
Memory usage of dataframe is 622.81 MB
Memory usage after optimization is: 259.50 MB
Decreased by 58.3%
Memory usage of dataframe is 32.85 MB
Memory usage after optimization is: 20.24 MB
Decreased by 38.4%
PRINTING OUT ALL NEW FEATURES IN THE DATASET

Features/Columns in balance_history include:  ['customer_id', 'credit_bureau_id', 'credit_status', 'credit_currency', 'days_since_credit_start', 'current_overdue_days', 'total_credit_amount', 'current_debt_amount', 'current_overdue_amount', 'months_on_time', 'months_1_30_dpd', 'months_31_60_dpd', 'months_61_90_dpd', 'months_91_120_dpd', 'months_120_plus_dpd', 'months_closed', 'months_no_info']

Features/Columns i

In [70]:
print("\nOptimizing datasets...")

# --- Credit Bureau History Fix ---
# Convert status columns to int16
status_cols = [col for col in balance_history.columns if col.startswith('months_')]
balance_history[status_cols] = balance_history[status_cols].fillna(0).astype(np.int16)

# --- Credit Card Transactions Optimization ---
# Convert to absolute values and optimize
for col in ['total_spent_amount', 'minimum_payment_amount', 'total_payment_amount']:
    transaction_history[col] = transaction_history[col].abs().astype(np.float32)

# --- Loan Repayment Optimization ---
# Calculate payment delay
loan_repayment_history['payment_delay_days'] = (
    loan_repayment_history['payment_date_days'] - loan_repayment_history['due_date_days']
).astype(np.float32)

# --- Customer Demographics Optimization ---
# Convert employment days to years
customer_bio_data['employment_years'] = (
    -customer_bio_data['employment_days'] / 365
).clip(0, 50).round(1).astype(np.float16)
customer_bio_data.drop('employment_days', axis=1, inplace=True)

# Save optimized datasets
balance_history.to_csv('balance_history.csv', index=False)
transaction_history.to_csv('transaction_history.csv', index=False)
loan_application_history.to_csv('loan_application_history.csv', index=False)
loan_repayment_history.to_csv('loan_repayment_history.csv', index=False)
customer_bio_data.to_csv('customer_bio_data.csv', index=False)

print("Optimization complete!")


Optimizing datasets...
Optimization complete!


In [71]:
from tqdm import tqdm

print("\nGenerating mobile_usage_data.csv...")

# Load necessary data
customer_bio_data = pd.read_csv('customer_bio_data.csv')
loan_app_history = pd.read_csv('loan_application_history.csv')

# Get target values per customer
target_per_customer = loan_app_history.groupby('customer_id')['repayment_capability'].first().reset_index()
mobile_data = customer_bio_data[['customer_id']].merge(
    target_per_customer, on='customer_id', how='left'
)

# Fill any missing targets with 0 (non-default)
mobile_data['repayment_capability'] = mobile_data['repayment_capability'].fillna(0)

# Set observation period parameters
OBSERVATION_DAYS = 30
SECONDS_PER_DAY = 86400
NIGHT_HOURS = (22, 6)
WORK_HOURS = (9, 17)

np.random.seed(42)

def generate_customer_mobile_data(target):
    """Generate realistic mobile usage data for a single customer"""
    data = {}
    target = int(target)
    
    # Core call metrics with risk-based adjustments
    base_call_count = max(10, np.random.poisson(80 - 25*target))
    call_duration_base = max(30, 180 - 60*target)
    
    data['total_calls_count'] = base_call_count
    data['total_call_duration_seconds'] = call_duration_base * base_call_count * abs(np.random.lognormal(0, 0.2))
    
    # Incoming/outgoing split
    outgoing_ratio = np.clip(0.4 + 0.3*target, 0.1, 0.9)
    data['outgoing_calls_count'] = int(base_call_count * outgoing_ratio)
    data['incoming_calls_count'] = base_call_count - data['outgoing_calls_count']
    
    # SMS metrics
    base_sms_count = max(5, np.random.poisson(50 + 30*target))
    data['total_sms_count'] = base_sms_count
    sms_out_ratio = np.clip(0.3 + 0.4*target, 0.1, 0.9)
    data['outgoing_sms_count'] = int(base_sms_count * sms_out_ratio)
    data['incoming_sms_count'] = base_sms_count - data['outgoing_sms_count']
    
    # Contact diversity
    data['unique_contacts_called'] = max(1, np.random.poisson(35 - 15*target))
    data['unique_sms_contacts'] = max(1, np.random.poisson(30 - 12*target))
    
    # Airtime behavior (high-risk: smaller, more frequent topups)
    topup_count = max(1, np.random.poisson(8 + 12*target))
    avg_topup = np.random.uniform(3, 10) if target == 1 else np.random.uniform(8, 25)
    
    data['airtime_topup_count'] = topup_count
    data['airtime_topup_total_amount'] = avg_topup * topup_count
    data['average_topup_amount'] = avg_topup
    
    # App usage patterns
    data['app_usage_count'] = max(10, np.random.poisson(100 + 50*target))
    data['app_usage_duration_seconds'] = abs(np.random.lognormal(6 + 0.5*target, 0.3)) * 3600
    data['unique_apps_used'] = max(1, np.random.poisson(25 - 8*target))
    
    # Time-based patterns (riskier behavior at night)
    night_call_ratio = np.clip(0.1 + 0.25*target, 0.05, 0.8)
    night_app_ratio = np.clip(0.15 + 0.3*target, 0.05, 0.8)
    
    data['night_time_calls_count'] = int(base_call_count * night_call_ratio)
    data['day_time_calls_count'] = base_call_count - data['night_time_calls_count']
    data['night_time_app_usage_seconds'] = data['app_usage_duration_seconds'] * night_app_ratio
    
    # Behavioral variability (higher for defaulters)
    data['call_duration_variance'] = 1000 + 2500*target * abs(np.random.chisquare(2))
    data['sms_count_variance'] = 50 + 150*target * abs(np.random.exponential(1))
    
    # Interaction patterns
    data['average_inter_call_time_seconds'] = OBSERVATION_DAYS * SECONDS_PER_DAY / (base_call_count + 1)
    data['average_inter_sms_time_seconds'] = OBSERVATION_DAYS * SECONDS_PER_DAY / (base_sms_count + 1)
    
    # Communication ratios
    data['call_out_in_ratio'] = (
        data['outgoing_calls_count'] / data['incoming_calls_count']
    ) if data['incoming_calls_count'] > 0 else 1.0
    
    data['sms_out_in_ratio'] = (
        data['outgoing_sms_count'] / data['incoming_sms_count']
    ) if data['incoming_sms_count'] > 0 else 1.0
    
    # Call duration extremes
    data['max_call_duration_seconds'] = call_duration_base * abs(np.random.lognormal(1.5, 0.3))
    data['min_call_duration_seconds'] = max(1, call_duration_base * abs(np.random.lognormal(-1, 0.4)))
    
    # Weekend patterns (riskier behavior on weekends)
    weekend_ratio = np.clip(0.3 + 0.4*target, 0.1, 0.7)
    data['weekend_call_count'] = int(base_call_count * weekend_ratio)
    data['weekday_call_count'] = base_call_count - data['weekend_call_count']
    
    # Daily averages
    data['daily_calls_average'] = base_call_count / OBSERVATION_DAYS
    data['daily_sms_average'] = base_sms_count / OBSERVATION_DAYS
    
    # App session metrics
    session_ratio = min(0.99, max(0.01, np.random.beta(1, 5 - 3*target)))
    data['longest_app_session_seconds'] = session_ratio * data['app_usage_duration_seconds']
    
    # App categories with risk-based preferences
    app_categories = ['Finance', 'Productivity', 'Communication', 
                     'Entertainment', 'Social', 'Gaming', 'Utilities']
    app_probs_low = [0.25, 0.20, 0.15, 0.10, 0.10, 0.10, 0.10]
    app_probs_high = [0.10, 0.10, 0.15, 0.25, 0.15, 0.15, 0.10]
    
    probs = app_probs_high if target == 1 else app_probs_low
    data['frequent_app_category'] = np.random.choice(app_categories, p=probs)
    
    return data

# Generate mobile data for all customers
rows = []
for idx, row in tqdm(mobile_data.iterrows(), total=len(mobile_data)):
    customer_data = generate_customer_mobile_data(row['repayment_capability'])
    customer_data['customer_id'] = row['customer_id']
    rows.append(customer_data)

mobile_df = pd.DataFrame(rows)
mobile_df = reduce_memory_usage(mobile_df)
mobile_df.to_csv('mobile_usage_data.csv', index=False)

print("Mobile data generation complete!")
print("All 6 datasets created successfully!")


Generating mobile_usage_data.csv...


100%|████████████████████████████████████████████████████████████████████████| 307511/307511 [02:28<00:00, 2069.86it/s]


Memory usage of dataframe is 77.42 MB
Memory usage after optimization is: 20.53 MB
Decreased by 73.5%
Mobile data generation complete!
All 6 datasets created successfully!


# CLEANING

In [78]:
# ------------------- CUSTOMER BIO DATA CLEANING -------------------
import pandas as pd
import numpy as np

print("Loading customer_bio_data.csv...")
bio_data = pd.read_csv('customer_bio_data.csv')
print(f"Initial shape: {bio_data.shape}")
print(f"Initial missing values:\n{bio_data.isnull().sum()}")

# 1. Occupation-specific handling (from EDA)
print("\n=== Occupation Handling ===")
bio_data['occupation'] = bio_data['occupation'].fillna('Unknown')

# Create occupation risk categories based on EDA
high_risk_occupations = ['Low-skill Laborers', 'Drivers', 'Waiters/barmen staff', 'Security staff', 'Cooking staff']
bio_data['occupation_risk'] = bio_data['occupation'].apply(
    lambda x: 'High' if x in high_risk_occupations else ('Medium' if x != 'Unknown' else 'Unknown')
)
print("Added occupation risk categories")

# 2. External scores handling
print("\n=== External Scores Handling ===")
for i in range(1, 4):
    col = f'external_score_{i}'
    bio_data[col] = bio_data[col].fillna(bio_data[col].median())
    
    # Create risk categories based on EDA
    bio_data[f'{col}_risk'] = pd.cut(
        bio_data[col],
        bins=[0, 0.4, 0.6, 1],
        labels=['High Risk', 'Medium Risk', 'Low Risk'],
        include_lowest=True
    )

# 3. Gender cleaning (from EDA)
print("\n=== Gender Handling ===")
bio_data = bio_data[bio_data['gender'].isin(['M', 'F'])]  # Remove XNA
gender_income_median = bio_data.groupby('gender')['annual_income'].median()
print(f"Median income by gender:\n{gender_income_median}")

# 4. Income categorization (from EDA)
print("\n=== Income Categorization ===")
income_bins = [0, 100000, 200000, 300000, 500000, float('inf')]
income_labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']
bio_data['income_category'] = pd.cut(bio_data['annual_income'], bins=income_bins, labels=income_labels)

# 5. Education handling (from EDA)
print("\n=== Education Handling ===")
education_risk = {
    'Lower secondary': 'High',
    'Secondary / secondary special': 'Medium',
    'Incomplete higher': 'Medium',
    'Higher education': 'Low',
    'Academic degree': 'Very Low'
}
bio_data['education_risk'] = bio_data['education_level'].map(education_risk)

# 6. Outlier treatment
print("\n=== Outlier Treatment ===")
bio_data['children_count'] = bio_data['children_count'].clip(upper=6)  # Based on EDA
bio_data['annual_income'] = bio_data['annual_income'].clip(upper=472500)  # 99th percentile

# 7. Save cleaned data
bio_data.to_csv('cleaned_customer_bio_data.csv', index=False)
print("\n✅ Cleaned data saved: cleaned_customer_bio_data.csv")

# ------------------- BALANCE HISTORY CLEANING -------------------
print("\nLoading balance_history.csv...")
balance = pd.read_csv('balance_history.csv')
print(f"Initial shape: {balance.shape}")
print(f"Initial missing values:\n{balance.isnull().sum()}")

# 1. Currency-specific handling (from EDA)
print("\n=== Currency Handling ===")
balance['credit_currency'] = balance['credit_currency'].fillna('currency_1')

# Create currency risk based on EDA default rates
currency_risk = {
    'currency_1': 'Medium',
    'currency_2': 'Low',
    'currency_3': 'High',
    'currency_4': 'Very Low'
}
balance['currency_risk'] = balance['credit_currency'].map(currency_risk)

# 2. Debt handling
print("\n=== Debt Handling ===")
balance['current_debt_amount'] = balance['current_debt_amount'].fillna(0)
balance['current_overdue_amount'] = balance['current_overdue_amount'].fillna(0)

# 3. Delinquency features
print("\n=== Delinquency Features ===")
del_cols = [c for c in balance.columns if c.startswith('months_') and '_dpd' in c]
balance['total_delinquent_months'] = balance[del_cols].sum(axis=1)
balance['severe_delinquency'] = (balance['months_91_120_dpd'] + balance['months_120_plus_dpd'] > 0).astype(int)

# 4. Outlier treatment
print("\n=== Outlier Treatment ===")
for col in ['total_credit_amount', 'current_debt_amount']:
    balance[col] = balance[col].clip(upper=balance[col].quantile(0.95))

# 5. Save cleaned data
balance.to_csv('cleaned_balance_history.csv', index=False)
print("\n✅ Cleaned data saved: cleaned_balance_history.csv")

# ------------------- TRANSACTION HISTORY CLEANING -------------------
print("\nLoading transaction_history.csv...")
trans = pd.read_csv('transaction_history.csv')
print(f"Initial shape: {trans.shape}")
print(f"Initial missing values:\n{trans.isnull().sum()}")

# 1. Payment handling
print("\n=== Payment Handling ===")
trans['minimum_payment_amount'] = trans['minimum_payment_amount'].fillna(0)

# 2. Risk features
print("\n=== Risk Features ===")
trans['credit_utilization'] = trans['current_balance'] / trans['credit_limit'].replace(0, np.nan)
trans['missed_payment'] = (trans['days_past_due'] > 0).astype(int)
trans['severe_delinquency'] = (trans['days_past_due_90_plus'] > 0).astype(int)

# 3. Outlier treatment
print("\n=== Outlier Treatment ===")
for col in ['total_spent_amount', 'credit_limit', 'current_balance']:
    trans[col] = trans[col].clip(upper=trans[col].quantile(0.99))

# 4. Save cleaned data
trans.to_csv('cleaned_transaction_history.csv', index=False)
print("\n✅ Cleaned data saved: cleaned_transaction_history.csv")

# ------------------- LOAN APPLICATION HISTORY CLEANING -------------------
print("\nLoading loan_application_history.csv...")
loans = pd.read_csv('loan_application_history.csv')
print(f"Initial shape: {loans.shape}")
print(f"Initial missing values:\n{loans.isnull().sum()}")

# 1. Handle missing values
print("\n=== Handling Missing Values ===")
loans['reject_reason_code'] = loans['reject_reason_code'].fillna('Approved')
loans['annuity_amount'] = loans['annuity_amount'].fillna(loans['annuity_amount'].median())
loans['approved_amount'] = loans['approved_amount'].fillna(loans['requested_amount'])

# 2. Risk features
print("\n=== Risk Features ===")
loans['approval_ratio'] = loans['approved_amount'] / loans['requested_amount'].replace(0, np.nan)
loans['was_rejected'] = (loans['application_status'] == 'Refused').astype(int)

# 3. Save cleaned data
loans.to_csv('cleaned_loan_application_history.csv', index=False)
print("\n✅ Cleaned data saved: cleaned_loan_application_history.csv")

# ------------------- LOAN REPAYMENT HISTORY CLEANING -------------------
print("\nLoading loan_repayment_history.csv...")
repayments = pd.read_csv('loan_repayment_history.csv')
print(f"Initial shape: {repayments.shape}")
print(f"Initial missing values:\n{repayments.isnull().sum()}")

# 1. Handle missing payments
print("\n=== Handling Missing Payments ===")
repayments['payment_date_days'] = repayments['payment_date_days'].fillna(repayments['due_date_days'])
repayments['paid_amount'] = repayments['paid_amount'].fillna(0)  # Assume no payment

# 2. Payment behavior features
print("\n=== Payment Behavior ===")
repayments['payment_delay'] = repayments['payment_date_days'] - repayments['due_date_days']
repayments['is_late'] = (repayments['payment_delay'] > 0).astype(int)
repayments['payment_completeness'] = repayments['paid_amount'] / repayments['due_amount'].replace(0, np.nan)

# 3. Save cleaned data
repayments.to_csv('cleaned_loan_repayment_history.csv', index=False)
print("\n✅ Cleaned data saved: cleaned_loan_repayment_history.csv")

# ------------------- MOBILE USAGE DATA CLEANING -------------------
print("\nLoading mobile_usage_data.csv...")
mobile = pd.read_csv('mobile_usage_data.csv')
print(f"Initial shape: {mobile.shape}")
print(f"Initial missing values:\n{mobile.isnull().sum()}")

# 1. Risk score based on EDA
print("\n=== Mobile Risk Score ===")
mobile['night_ratio'] = mobile['night_time_calls_count'] / (mobile['day_time_calls_count'] + 1)
mobile['gaming_entertainment'] = mobile['frequent_app_category'].isin(['Gaming', 'Entertainment']).astype(int)
mobile['risk_score'] = (
    0.4 * mobile['night_ratio'] +
    0.3 * (mobile['airtime_topup_count'] > 10).astype(int) +
    0.3 * mobile['gaming_entertainment']
)

# 2. Outlier treatment
print("\n=== Outlier Capping ===")
for col in ['app_usage_duration_seconds', 'total_call_duration_seconds', 'longest_app_session_seconds']:
    mobile[col] = mobile[col].clip(upper=mobile[col].quantile(0.99))

# 3. Save cleaned data
mobile.to_csv('cleaned_mobile_usage_data.csv', index=False)
print("\n✅ Cleaned data saved: cleaned_mobile_usage_data.csv")

Loading customer_bio_data.csv...
Initial shape: (307511, 14)
Initial missing values:
customer_id                0
gender                     0
owns_car_flag              0
owns_property_flag         0
children_count             0
annual_income              0
education_level            0
family_status              0
age_in_days                0
occupation             96391
external_score_1      173378
external_score_2         660
external_score_3       60965
employment_years           0
dtype: int64

=== Occupation Handling ===
Added occupation risk categories

=== External Scores Handling ===

=== Gender Handling ===
Median income by gender:
gender
F    135000.0
M    180000.0
Name: annual_income, dtype: float64

=== Income Categorization ===

=== Education Handling ===

=== Outlier Treatment ===

✅ Cleaned data saved: cleaned_customer_bio_data.csv

Loading balance_history.csv...
Initial shape: (1716428, 17)
Initial missing values:
customer_id                     0
credit_bureau_id     